In [2]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
from torchtext.vocab import build_vocab_from_iterator
eng = spacy.load("en_core_web_md") # Load the English model to tokenize English text
de = spacy.load("de_core_news_md") # Load the German model to tokenize German text

In [2]:
FILE_PATH = 'data/deu.txt'
data_pipe = dp.iter.IterableWrapper([FILE_PATH])  # note the file_path is a list
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = data_pipe.parse_csv(skip_lines=0,
                                delimiter='\t',
                                as_tuple=True)
# Process of opening the file is different than the usual case

In [5]:
for sample in data_pipe:
    print(sample)
    break

('Go.', 'Geh.')


In [ ]:
list(data_pipe)[799:850]

In [4]:
def removeAttribution(row):
    """
    Function to keep the first two elements in a tuple
    """
    return row[:2]
data_pipe = data_pipe.map(removeAttribution)

In [6]:
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def deTokenize(text):
    """
    Tokenize a German text and return a list of tokens
    """
    return [token.text for token in de.tokenizer(text)]

In [7]:
print(engTokenize("Have a good day!!!"))
print(deTokenize("Haben Sie einen guten Tag!!!"))

['Have', 'a', 'good', 'day', '!', '!', '!']
['Haben', 'Sie', 'einen', 'guten', 'Tag', '!', '!', '!']


In [8]:
def getTokens(data_iter, place):
    """
    Function to yield tokens from an iterator. Since, our iterator contains
    tuple of sentences (source and target), `place` parameters defines for which
    index to return the tokens for. `place=0` for source and `place=1` for target
    """
    for english, german in data_iter:
        if place == 0:
            yield engTokenize(english)
        else:
            yield deTokenize(german)

- sos: for start of sentence

- eos: for end of sentence

- unk: for unknown words. An example of unknown word is the one skipped because of min_freq=2.

- pad: is the padding token. While training, a model we mostly train in batches. In a batch, there can be sentences of different length. So, we pad the shorter sentences with <pad> token to make length of all sequences in the batch equal.

In [9]:
source_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,0),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
source_vocab.set_default_index(source_vocab['<unk>'])

At line 5, we set special_first=True. Which means <pad> will get index 0, <sos> index 1, <eos> index 2, and <unk> will get index 3 in the vocabulary.

At line 7, we set default index as index of <unk>. That means if some word is not in vocabulary, we will use <unk> instead of that unknown word.

In [10]:
target_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,1),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
target_vocab.set_default_index(target_vocab['<unk>'])

In [11]:
print(source_vocab.get_itos()[:9])
print(target_vocab.get_itos()[:9])

['<pad>', '<sos>', '<eos>', '<unk>', '.', 'I', 'Tom', 'to', 'you']
['<pad>', '<sos>', '<eos>', '<unk>', '.', ',', 'Tom', 'Ich', '?']


In [ ]:
T.VocabTransform(vocab=source_vocab)(['wipes', '<sos>'])

In [12]:
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
        ## Add <sos> at beginning of each sentence. 
        # 1 because the index for <sos> in vocabulary is
        T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence.
        # 2 because the index for <eos> in vocabulary is
        T.AddToken(2, begin=False)
    )
    return text_tranform

In [14]:
temp_list = list(data_pipe)
some_sentence = temp_list[799][0]
print("Some sentence=", end="")
print(some_sentence)


Some sentence=I fainted.


In [19]:
# getTransform function is not required, a simple declaiton of transform will be 
# sufficient
transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))
print("Transformed sentence=", end="")
print(transformed_sentence)  # Transformed sentence=[1, 5, 2897, 4, 2]

Transformed sentence=[1, 5, 2897, 4, 2]


In [20]:
index_to_string = source_vocab.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")  # <sos> I fainted . <eos> 

<sos> I fainted . <eos> 

In [21]:
def applyTransform(sequence_pair):
    """
    Apply transforms to sequence of tokens in a sequence pair
    """

    return (
        getTransform(source_vocab)(engTokenize(sequence_pair[0])),
        getTransform(target_vocab)(deTokenize(sequence_pair[1]))
    )
data_pipe = data_pipe.map(applyTransform) ## Apply the function to each element in the iterator
temp_list = list(data_pipe)
print(temp_list[0])

([1, 616, 4, 2], [1, 739, 4, 2])


In [22]:
def sortBucket(bucket):
    """
    Function to sort a given bucket. Here, we want to sort based on the length of
    source and target sequence.
    """
    return sorted(bucket, key=lambda x: (len(x[0]), len(x[1])))

In [23]:
data_pipe = data_pipe.bucketbatch(
    batch_size = 4, batch_num=5,  bucket_num=1,
    use_in_batch_shuffle=False, sort_key=sortBucket
)

We keep batch size = 4.

batch_num is the number of batches to keep in a bucket

bucket_num is the number of buckets to keep in a pool for shuffling

sort_key specifies the function that takes a bucket and sorts it

In [24]:
def separateSourceTarget(sequence_pairs):
    """
    input of form: `[(X_1,y_1), (X_2,y_2), (X_3,y_3), (X_4,y_4)]`
    output of form: `((X_1,X_2,X_3,X_4), (y_1,y_2,y_3,y_4))`
    """
    sources,targets = zip(*sequence_pairs)
    return sources,targets

## Apply the function to each element in the iterator
data_pipe = data_pipe.map(separateSourceTarget)
print(list(data_pipe)[0])

(([1, 5, 964, 4, 2], [1, 5, 258, 4, 2], [1, 5, 360, 4, 2], [1, 5335, 21, 4, 2]), ([1, 7, 22, 1475, 4, 2], [1, 7, 22, 376, 4, 2], [1, 297, 10, 19, 561, 4, 2], [1, 896, 32, 21, 33, 1133, 24, 2]))


In [25]:
def applyPadding(pair_of_sequences):
    """
    Convert sequences to tensors and apply padding
    """
    return (T.ToTensor(0)(list(pair_of_sequences[0])), T.ToTensor(0)(list(pair_of_sequences[1])))
## `T.ToTensor(0)` returns a transform that converts the sequence to `torch.tensor` and also applies
# padding. Here, `0` is passed to the constructor to specify the index of the `<pad>` token in the
# vocabulary.
data_pipe = data_pipe.map(applyPadding)

In [26]:
source_index_to_string = source_vocab.get_itos()
target_index_to_string = target_vocab.get_itos()

def showSomeTransformedSentences(data_pipe):
    """
    Function to show how the sentences look like after applying all transforms.
    Here we try to print actual words instead of corresponding index
    """
    for sources,targets in data_pipe:
        if sources[0][-1] != 0:
            continue # Just to visualize padding of shorter sentences
        for i in range(4):
            source = ""
            for token in sources[i]:
                source += " " + source_index_to_string[token]
            target = ""
            for token in targets[i]:
                target += " " + target_index_to_string[token]
            print(f"Source: {source}")
            print(f"Traget: {target}")
        break

showSomeTransformedSentences(data_pipe)

Source:  <sos> <unk> . <eos> <pad>
Traget:  <sos> Fang an . <eos>
Source:  <sos> Do it . <eos>
Traget:  <sos> Mache es ! <eos>
Source:  <sos> Do it . <eos>
Traget:  <sos> Tue es . <eos>
Source:  <sos> Go on . <eos>
Traget:  <sos> Mach weiter . <eos>
